### `MaxPool2D` code

In [1]:
import numpy as np
import torch # needed for tests
from tqdm import tqdm # needed for tests

In [61]:
class MaxPool2D:
    ''' Computes maxpool given the input parameters '''
    
    ''' * The class implementation will be along the lines of torch.nn.MaxPool2d in order to 
          enable comparison of this NumPy only implementation and seamless testing
    '''
    '''
        TODO:
        * Replace `torch.round()` with `np.allclose()` for tests
        * Optimizing code
    '''
    
    def __init__(
        self, 
        kernel_size, 
        padding = (0, 0), 
        stride = (1, 1), 
        dilation = (1, 1), 
        return_indices = False,
        verbose = False
        ):
        super(MaxPool2D, self).__init__()
        
        ''' mandatory parameters '''
        if isinstance(kernel_size, tuple):
            self.kernel_size = kernel_size
        elif isinstance(kernel_size, int):
            self.kernel_size = (kernel_size, kernel_size)
        else:
            raise Exception('invalid input parameters: kernel_size should either be an int or a tuple')
        
        ''' optional parameters '''
        if isinstance(padding, str):
            if padding == 'valid':
                self.padding = (0, 0)
            elif padding == 'same':
                raise Exception("invalid input parameters: padding = 'same' not yet supported")
            else:
                raise Exception('invalid input parameters: padding is not valid')
        elif isinstance(padding, tuple):
            if padding[0] >= 0 and padding[1] >= 0:
                self.padding = padding
            else:
                raise Exception('invalid input parameters: padding is not valid')
        elif isinstance(padding, int):
            if padding >= 0:
                self.padding = (padding, padding)
            else:
                raise Exception('invalid input parameters: padding is not valid')
        else:
            raise Exception('invalid input parametersL padding should be either an int or a tuple')
        if isinstance(stride, tuple):
            if stride[0] >= 1 and stride[1] >= 1:
                self.stride = stride
            else:
                raise Exception('invalid input parameters: stride is not valid')
        elif isinstance(stride, int):
            if stride >= 1:
                self.stride = (stride, stride)
            else:
                raise Exception('invalid input parameters: stride is not valid')
        else:
            raise Exception('invalid input parameters: stride should be either an int or a tuple')
        if isinstance(dilation, tuple):
            if dilation[0] >= 1 and dilation[1] >= 1:
                self.dilation = dilation
            else:
                raise Exception('invalid input parameters: dilation is not valid')
        elif isinstance(dilation, int):
            if dilation >= 1:
                self.dilation = (dilation, dilation)
            else:
                raise Exception('invalid input parameters: dilation is not valid')
        else:
            raise Exception('invalid input parameters: dilation should be either an int or a tuple')
        self.return_indices = return_indices
        
        ''' optional parameters (dummy, yet to be implemented)'''
        
        
        ''' additional parameters (different from torch.nn.Conv2D)'''
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        self.verboseprint('*** parameters ***')
        self.verboseprint('kernel_size: {}'.format(self.kernel_size))
        self.verboseprint('padding: {}, stride: {}, dilation factor: {}'.format(self.padding, self.stride, self.dilation))
        self.verboseprint('\n')
    
    def forward(self, _input):
        ''' forward pass to perform convolution '''
        
        ''' do error checking '''
        _input_n, _input_c, _input_h, _input_w = _input.shape
        if _input_h + 2 * self.padding[0] < self.dilation[0] * (self.kernel_size[0] - 1) + 1: # check if (dilated) ker_h is valid
            raise Exception('invalid input parameters: kernel height is larger than input height')
        if _input_w + 2 * self.padding[1] < self.dilation[1] * (self.kernel_size[1] - 1) + 1: # check if (dilated) ker_w is valid
            raise Exception('invalid input parameters: kernel width is larger than input width')
        if ((_input_h + 2 * self.padding[0] - (self.dilation[0] * (self.kernel_size[0] - 1) + 1)) / self.stride[0]) + 1 < 0: # check if out_h is valid
            raise Exception('invalid input parameters: output height is negative')
        if ((_input_w + 2 * self.padding[1] - (self.dilation[1] * (self.kernel_size[1] - 1) + 1)) / self.stride[1]) + 1 < 0: # check if out_w is valid
            raise Exception('invalid input parameters: output width is negative')
        if self.padding[0] > self.kernel_size[0] // 2: # as PyTorch mandates this
            raise Exception('invalid input parameters: padding height is larger than half of kernel height')
        if self.padding[1] > self.kernel_size[1] // 2: # as PyTorch mandates this
            raise Exception('invalid input parameters: padding width is larger than half of kernel width')
        
        ''' add zero padding based on the input parameters '''
        if self.padding != (0, 0):
            _input = np.array([[np.pad(channel, ((self.padding[0], self.padding[0]), (self.padding[1], self.padding[1])), 'constant', constant_values = -np.inf) for channel in batch] for batch in _input])    
            self.verboseprint('*** padded input image ***')
            self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(_input.shape[0], _input.shape[1], _input.shape[2], _input.shape[3]))
            self.verboseprint(_input)
            self.verboseprint('\n')
        
        ''' use the provided kernels or create random kernels based on the input kernel parameters '''
        kernels = []
        self.verboseprint('*** kernels ***')
        self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(_input_c, 1, self.kernel_size[0], self.kernel_size[1]))
        for k in range(_input_c):
            kernel = np.ones((self.kernel_size[0], self.kernel_size[1])) # define a kernel of 1s based on the kernel parameters
            kernels.append(kernel)
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(kernel)
        self.verboseprint('\n')
            
        ''' dilate a kernel '''
        dil_ker_h = self.dilation[0] * (self.kernel_size[0] - 1) + 1
        dil_ker_w = self.dilation[1] * (self.kernel_size[1] - 1) + 1
        dil_kernels = []
        for kernel in kernels:
            dil_kernel = -np.inf * np.ones((dil_ker_h, dil_ker_w)) # instantiate a dilated kernel of all -np.inf (holes)
            for row in range(len(kernel)):
                for col in range(len(kernel[0])):
                    dil_kernel[self.dilation[0] * row][self.dilation[1] * col] = kernel[row][col]
            dil_kernels.append(dil_kernel.tolist())
        kernels, self.kernel_size = dil_kernels, (dil_ker_h, dil_ker_w)
        self.verboseprint('*** dilated kernels ***')
        self.verboseprint('dilation factor: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.dilation, _input_c, self.kernel_size[0], self.kernel_size[1]))
        for k in range(_input_c):
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(kernels[k])
        self.verboseprint('\n')
        
        ''' compute output volume from the input and kernel parameters '''
        _input_n, _input_c, _input_h, _input_w = _input.shape
        out_n = int(_input_n)
        out_c = int(_input_c)
        out_h = int((_input_h - self.kernel_size[0]) / self.stride[0]) + 1
        out_w = int((_input_w - self.kernel_size[1]) / self.stride[1]) + 1
        output = np.zeros([out_n, out_c, out_h, out_w])
        max_indices = np.zeros([out_n, out_c, out_h, out_w], dtype = int)
        
        ''' parse through every element of the output and compute the convolution value for that element '''
        for b_out in range(out_n):
            for c_out in range(out_c):
                for h_out in range(out_h):
                    for w_out in range(out_w):
                        # convolve kernel over the input slices
                        self.verboseprint('kernel indices, image indices')
                        self.verboseprint('[n, c, h, w]', '[n, c, h, w]')
                        convol_sum = -np.inf
                        max_index = 0
                        ker_h = self.kernel_size[0]
                        ker_w = self.kernel_size[1]
                        for h_ker in range(ker_h):
                            for w_ker in range(ker_w):
                                self.verboseprint([c_out, h_ker, w_ker], [b_out, c_out, h_ker + self.stride[0] * h_out, w_ker + self.stride[1] * w_out])
                                if kernels[c_out][h_ker][w_ker] != -np.inf:
                                    val = kernels[c_out][h_ker][w_ker] * _input[b_out][c_out][h_ker + self.stride[0] * h_out][w_ker + self.stride[1] * w_out]
                                    if val > convol_sum:
                                        convol_sum = val
                                        max_index = (h_ker + self.stride[0] * h_out) * _input_h + (w_ker + self.stride[1] * w_out)
                        self.verboseprint('\n')
                        output[b_out, c_out, h_out, w_out] += convol_sum
                        max_indices[b_out, c_out, h_out, w_out] += max_index
        self.verboseprint('*** MaxPool2D output ***')
        output_shape = output.shape
        self.verboseprint('output batches: {}, ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2], output_shape[3]))
        assert((out_n, out_c, out_h, out_w) == output_shape)
        self.verboseprint(output)
        self.verboseprint('\n')
        if self.return_indices:
            return (output, max_indices)
        return output

### Standalone test (random input), where PyTorch output is (supposedly) incorrect

In [62]:
in_channels = 1 # input channels
kernel_size = (2, 2) # kernel size

padding = (1, 1) # padding (optional)
stride = (1, 1) # stride (optional)
dilation = (2, 2) # dilation factor (optional)

in_batches = 1 # input batches
in_h = 1 # input height
in_w = 2 # input weight

_input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters

In [63]:
# get MaxPool2D output with the random inputs

maxpool2d = MaxPool2D(kernel_size, stride = stride, padding = padding, dilation = dilation) # call an instance of the class with the input parameters 
_output = maxpool2d.forward(_input) # perform maxpooling
print("*** MaxPool2D output ***")
print(_output)

*** MaxPool2D output ***
[[[[-inf -inf]]]]


In [64]:
# get PyTorch output with the same random inputs as above

x = torch.DoubleTensor(_input)
m = torch.nn.MaxPool2d(kernel_size, stride = stride, padding = padding, dilation = dilation)
output = m(x)
print("*** PyTorch output ***")
print(output)

*** PyTorch output ***
tensor([[[[-inf, -inf]]]], dtype=torch.float64)


In [65]:
# compare outputs of MaxPool2D and PyTorch
if isinstance(_output, tuple):
    print(torch.equal(torch.round(torch.DoubleTensor(_output[0])), torch.round(output[0])) 
          and torch.equal(torch.round(torch.DoubleTensor(_output[1])), torch.round(output[1]))) # check if both the output and max indices are correct
else:
    print(torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output))) # need to round the output due to precision difference

True


### Extensive tests (random kernel, random input)

In [66]:
def valid_params(num_tests):
    ''' generates `num_tests` number of valid input and kernel parameters '''
    np.random.seed(1729)
    params_list = []
    sample_count = 0
    while sample_count < num_tests:
        in_channels = np.random.randint(20) + 1 # input channels
        
        kernel_h = np.random.randint(20) + 1
        kernel_w = np.random.randint(20) + 1
        kernel_size = (kernel_h, kernel_w) # kernel size
        
        padding_h = np.random.randint(10) + 1
        padding_w = np.random.randint(10) + 1
        padding = (padding_h, padding_w) # padding (optional)
        stride_h = np.random.randint(5) + 1
        stride_w = np.random.randint(5) + 1
        stride = (stride_h, stride_w) # stride (optional)
        dilation_h = np.random.randint(10) + 1
        dilation_w = np.random.randint(10) + 1
        dilation = (dilation_h, dilation_w) # dilation factor (optional)
        
        in_batches = np.random.randint(5) + 1 # input batches
        in_h = np.random.randint(30) + 5 # input height
        in_w = np.random.randint(30) + 5 # input weight
        
        return_indices = np.random.choice([True, False])
        
        ker_h_flag, ker_w_flag, out_h_flag, out_w_flag, pad_h_flag, pad_w_flag = True, True, True, True, True, True
        
        if in_h + 2 * padding_h < dilation_h * (kernel_h - 1) + 1: # check if (dilated) ker_h is valid
            ker_h_flag = False
        if in_w + 2 * padding_w < dilation_w * (kernel_w - 1) + 1: # check if (dilated) ker_w is valid
            ker_w_flag = False
        if ((in_h + 2 * padding_h - (dilation_h * (kernel_h - 1) + 1)) / stride_h) + 1 < 0: # check if out_h is valid
            out_h_flag = False
        if ((in_w + 2 * padding_w - (dilation_w * (kernel_w - 1) + 1)) / stride_w) + 1 < 0: # check if out_w is valid
            out_w_flag = False
        if padding_h > kernel_h // 2: # as PyTorch mandates this
            pad_h_flag = False
        if padding_w > kernel_w // 2: # as PyTorch mandates this
            pad_w_flag = False
    
        if ker_h_flag and ker_w_flag and out_h_flag and out_w_flag and pad_h_flag and pad_w_flag:
            params_list.append({'in_channels': in_channels, 'kernel_size': kernel_size,
                          'padding': padding, 'stride': stride, 'dilation': dilation, 'in_batches': in_batches,
                          'in_h': in_h, 'in_w': in_w, 'return_indices': return_indices})
            sample_count += 1
    return params_list

In [67]:
def run_tests(num_tests):
    ''' sweep different input parameters and test by comparing outputs of Conv2D and PyTorch '''
    
    num_passed = 0
    params_list = valid_params(num_tests)
    print('Number of tests: {}\n\n'.format(len(params_list)))

    for i, params in enumerate(tqdm(params_list)):
        print('Test: {}\nParams: {}'.format(i, params))
        in_channels = params['in_channels'] # input channels
        kernel_size = params['kernel_size'] # kernel size

        padding = params['padding'] # padding (optional)
        stride = params['stride'] # stride (optional)
        dilation = params['dilation'] # dilation factor (optional)
        return_indices = return_indices = params['return_indices'] # return indices (optional)
        
        in_batches = params['in_batches'] # input batches
        in_h = params['in_h'] # input height
        in_w = params['in_w'] # input weight
        
        _input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
    
        try:
            # get MaxPool2D output with the random inputs
            maxpool2d = MaxPool2D(kernel_size, stride = stride, padding = padding, dilation = dilation, return_indices = return_indices) # call an instance of the class with the input parameters 
            _output = maxpool2d.forward(_input) # perform convolution

            # get PyTorch output with the same random inputs as above
            x = torch.DoubleTensor(_input)
            m = torch.nn.MaxPool2d(kernel_size, stride = stride, padding = padding, dilation = dilation, return_indices = return_indices)
            output = m(x)

        except Exception as e:
            print(e)
            print('Result: False\n\n') # treating exception as a failed test
            continue

        # compare outputs of MaxPool2D and PyTorch
        if isinstance(_output, tuple):
            result = torch.equal(torch.round(torch.DoubleTensor(_output[0])), torch.round(output[0])) # check if both the output and max indices are correct
            print('Result: {}\n\n'.format(result))
        else:
            result = torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output)) # need to round the output due to precision difference
            print('Result: {}\n\n'.format(result))
        if result:
            num_passed += 1
    print('{} out of {} ({}%) tests passed'.format(num_passed, num_tests, float(100 * num_passed / num_tests))) 

In [ ]:
num_tests = 100
run_tests(num_tests)

Number of tests: 100




  0%|                                                   | 0/100 [00:00<?, ?it/s]

Test: 0
Params: {'in_channels': 16, 'kernel_size': (20, 12), 'padding': (7, 4), 'stride': (4, 5), 'dilation': (1, 1), 'in_batches': 2, 'in_h': 15, 'in_w': 26, 'return_indices': True}


  1%|▍                                          | 1/100 [00:00<00:30,  3.20it/s]

Result: True


Test: 1
Params: {'in_channels': 18, 'kernel_size': (10, 20), 'padding': (2, 5), 'stride': (4, 4), 'dilation': (2, 1), 'in_batches': 1, 'in_h': 26, 'in_w': 14, 'return_indices': True}
Result: True


Test: 2
Params: {'in_channels': 3, 'kernel_size': (2, 19), 'padding': (1, 1), 'stride': (1, 3), 'dilation': (7, 1), 'in_batches': 5, 'in_h': 25, 'in_w': 32, 'return_indices': True}


...

Result: True


Test: 21
Params: {'in_channels': 1, 'kernel_size': (4, 17), 'padding': (1, 3), 'stride': (1, 5), 'dilation': (6, 1), 'in_batches': 5, 'in_h': 22, 'in_w': 17, 'return_indices': False}
Result: True


Test: 22
Params: {'in_channels': 19, 'kernel_size': (18, 9), 'padding': (9, 4), 'stride': (2, 3), 'dilation': (2, 1), 'in_batches': 2, 'in_h': 20, 'in_w': 17, 'return_indices': True}


 23%|█████████▋                                | 23/100 [00:14<01:09,  1.11it/s]

Result: True


Test: 23
Params: {'in_channels': 14, 'kernel_size': (8, 9), 'padding': (2, 2), 'stride': (4, 5), 'dilation': (2, 2), 'in_batches': 5, 'in_h': 28, 'in_w': 32, 'return_indices': True}


 24%|██████████                                | 24/100 [00:15<01:01,  1.23it/s]

Result: True


Test: 24
Params: {'in_channels': 10, 'kernel_size': (20, 12), 'padding': (10, 6), 'stride': (3, 5), 'dilation': (1, 2), 'in_batches': 2, 'in_h': 33, 'in_w': 34, 'return_indices': False}
